In [1]:
# Essential Libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For modeling (optional, if needed later)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Date and time handling
from datetime import datetime

# Suppress warnings for cleaner output
import os
import warnings
warnings.filterwarnings("ignore")

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")


In [1]:
# Enhanced Excel Data Loader with Error Handling and Logging
import pandas as pd
import os
import re
from pathlib import Path

def load_excel_sheets(excel_path, preview_rows=2, create_globals=True):
    """
    Load all sheets from an Excel file with enhanced error handling and logging.
    
    Parameters:
    -----------
    excel_path : str
        Path to the Excel file
    preview_rows : int
        Number of rows to preview for each sheet
    create_globals : bool
        Whether to create global DataFrame variables
    
    Returns:
    --------
    dict : Dictionary with lowercase sheet names as keys and DataFrames as values
    """
    
    df_lowercase = {}
    created_variables = []
    
    # Convert to Path object for better path handling
    file_path = Path(excel_path)
    
    # Check if file exists
    if not file_path.exists():
        print(f"❌ Error: File not found at: {excel_path}")
        print(f"Current working directory: {os.getcwd()}")
        return df_lowercase
    
    try:
        print(f"📂 Loading Excel file: {excel_path}")
        
        # Read all sheets
        df_dict = pd.read_excel(excel_path, sheet_name=None)
        
        # Convert sheet names to lowercase and clean them
        df_lowercase = {}
        for sheet_name, df in df_dict.items():
            clean_sheet_name = sheet_name.lower().strip()
            df_lowercase[clean_sheet_name] = df
        
        print(f"✅ Successfully loaded {len(df_lowercase)} sheets")
        
        # Preview and create variables
        print(f"\n{'='*80}")
        print(f"📊 SHEET PREVIEW AND VARIABLE CREATION")
        print(f"{'='*80}")
        
        for sheet_key, df_value in df_lowercase.items():
            print(f"\n📄 Sheet: '{sheet_key}'")
            print(f"   Shape: ({df_value.shape[0]:,} rows, {df_value.shape[1]:,} columns)")
            
            # Show column names
            if len(df_value.columns) <= 10:
                print(f"   Columns: {list(df_value.columns)}")
            else:
                print(f"   Columns: {list(df_value.columns[:5])} ... {list(df_value.columns[-2:])}")
            
            # Preview data
            if not df_value.empty:
                print(f"   Preview (first {preview_rows} rows):")
                print(df_value.head(preview_rows).to_string(index=False, max_cols=8))
            else:
                print("   ⚠️  Sheet is empty")
            
            # Create global variable if requested
            if create_globals:
                variable_name = create_safe_variable_name(sheet_key)
                globals()[variable_name] = df_value
                created_variables.append(variable_name)
                print(f"   ✅ Created variable: {variable_name}")
            
            print("-" * 60)
        
        # Summary
        print(f"\n📋 SUMMARY:")
        print(f"   • Total sheets loaded: {len(df_lowercase)}")
        print(f"   • Total rows across all sheets: {sum(df.shape[0] for df in df_lowercase.values()):,}")
        print(f"   • Created variables: {created_variables}")
        
        return df_lowercase
        
    except Exception as e:
        print(f"❌ Error loading Excel file: {str(e)}")
        return df_lowercase

def create_safe_variable_name(sheet_name):
    """
    Create a safe Python variable name from a sheet name.
    
    Parameters:
    -----------
    sheet_name : str
        Original sheet name
    
    Returns:
    --------
    str : Safe variable name
    """
    # Replace spaces and special characters with underscores
    safe_name = re.sub(r'[^a-zA-Z0-9_]', '_', sheet_name)
    
    # Remove consecutive underscores
    safe_name = re.sub(r'_+', '_', safe_name)
    
    # Remove leading/trailing underscores
    safe_name = safe_name.strip('_')
    
    # Ensure it starts with a letter or underscore (not a number)
    if safe_name and safe_name[0].isdigit():
        safe_name = 'sheet_' + safe_name
    
    # Add df_ prefix
    return f'df_{safe_name}' if safe_name else 'df_unnamed_sheet'

def get_sheet_info(df_dict):
    """
    Get summary information about all loaded sheets.
    
    Parameters:
    -----------
    df_dict : dict
        Dictionary of DataFrames
    
    Returns:
    --------
    pd.DataFrame : Summary information
    """
    if not df_dict:
        print("No sheets to summarize.")
        return pd.DataFrame()
    
    summary_data = []
    for sheet_name, df in df_dict.items():
        info = {
            'Sheet Name': sheet_name,
            'Rows': df.shape[0],
            'Columns': df.shape[1],
            'Memory Usage (KB)': round(df.memory_usage(deep=True).sum() / 1024, 2),
            'Has Missing Values': df.isnull().any().any(),
            'Numeric Columns': df.select_dtypes(include=['number']).shape[1],
            'Text Columns': df.select_dtypes(include=['object']).shape[1],
            'Date Columns': df.select_dtypes(include=['datetime']).shape[1]
        }
        summary_data.append(info)
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df.sort_values('Rows', ascending=False)

# Usage Example:
if __name__ == "__main__":
    # Define the Excel path
    excel_path = r'data\Project Assessment Data.xlsx'
    
    # Load the sheets
    df_lowercase = load_excel_sheets(excel_path, preview_rows=3, create_globals=True)
    
    # Get summary information
    if df_lowercase:
        print(f"\n{'='*80}")
        print("📈 DETAILED SHEET SUMMARY")
        print(f"{'='*80}")
        summary = get_sheet_info(df_lowercase)
        print(summary.to_string(index=False))
        
        # Show available DataFrames in globals
        df_variables = [var for var in globals() if var.startswith('df_') and isinstance(globals()[var], pd.DataFrame)]
        print(f"\n🎯 Available DataFrame variables: {df_variables}")

📂 Loading Excel file: data\Project Assessment Data.xlsx
✅ Successfully loaded 14 sheets

📊 SHEET PREVIEW AND VARIABLE CREATION

📄 Sheet: 'plant description'
   Shape: (13 rows, 2 columns)
   Columns: ['Plant Code', 'Name']
   Preview (first 3 rows):
Plant Code   Name
      A110 Plant1
      A111 Plant2
      A112 Plant3
   ✅ Created variable: df_plant_description
------------------------------------------------------------

📄 Sheet: 'afko'
   Shape: (200 rows, 183 columns)
   Columns: ['MANDT', 'AUFNR 2 Order Number', 'GLTRP', 'GSTRP', 'FTRMS'] ... ['QPGT.LTEXTV', 'QPGT.INAKTIV']
   Preview (first 3 rows):
 MANDT  AUFNR 2 Order Number    GLTRP    GSTRP  ...  QPGT.SPRACHE  QPGT.KURZTEXT  QPGT.LTEXTV  QPGT.INAKTIV
   600             340011104 20241005 20241005  ...           NaN            NaN          NaN           NaN
   600             340011105 20241005 20241005  ...           NaN            NaN          NaN           NaN
   600             340011106 20241005 20241005  ...           

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings

def clean_qmel_data(df_qmel, verbose=True):
    """
    Comprehensive cleaning pipeline for QMEL (Quality Notifications) data.
    
    Parameters:
    -----------
    df_qmel : pd.DataFrame
        Raw QMEL DataFrame
    verbose : bool
        Whether to print detailed progress information
    
    Returns:
    --------
    pd.DataFrame : Cleaned QMEL DataFrame
    dict : Cleaning summary statistics
    """
    
    if df_qmel is None or df_qmel.empty:
        print("❌ DataFrame is None or empty. Cannot proceed with cleaning.")
        return df_qmel, {}
    
    # Create a copy to avoid modifying the original
    df_clean = df_qmel.copy()
    initial_shape = df_clean.shape
    cleaning_stats = {
        'initial_rows': initial_shape[0],
        'initial_columns': initial_shape[1],
        'date_columns_converted': 0,
        'missing_values_before': df_clean.isnull().sum().sum(),
        'duplicates_removed': 0
    }
    
    if verbose:
        print("🧹 STARTING QMEL DATA CLEANING PIPELINE")
        print("=" * 60)
        print(f"Initial dataset shape: {initial_shape}")
    
    # 1. Initial Data Assessment
    if verbose:
        print(f"\n📊 1. INITIAL DATA ASSESSMENT")
        print(f"Shape: {df_clean.shape}")
        print(f"Total missing values: {df_clean.isnull().sum().sum():,}")
        print(f"Memory usage: {df_clean.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    # 2. Handle Missing Values Analysis
    if verbose:
        print(f"\n🔍 2. MISSING VALUES ANALYSIS")
        missing_analysis = analyze_missing_values(df_clean)
        if not missing_analysis.empty:
            print(missing_analysis.to_string())
        else:
            print("✅ No missing values found!")
    
    # 3. Date Column Conversion
    if verbose:
        print(f"\n📅 3. DATE COLUMN CONVERSION")
    
    # SAP date columns in QMEL
    date_columns = {
        'ERDAT': 'Creation Date',
        'AEDAT': 'Changed Date', 
        'PSTER': 'Period From',
        'PETRI': 'Period To',
        'BEZDT': 'Reference Date',
        'QMDAT': 'Notification Date',
        'LTRMN': 'Delivery Date'
    }
    
    date_conversion_results = {}
    
    for col, description in date_columns.items():
        if col in df_clean.columns:
            result = convert_sap_date_column(df_clean, col, verbose=verbose)
            date_conversion_results[col] = result
            if result['converted']:
                cleaning_stats['date_columns_converted'] += 1
        elif verbose:
            print(f"   ⚠️  Column '{col}' ({description}) not found")
    
    # 4. Data Type Optimization
    if verbose:
        print(f"\n🔧 4. DATA TYPE OPTIMIZATION")
    
    # Convert numeric columns that might be stored as strings
    numeric_candidates = ['PRIOK', 'QMNUM', 'MATNR', 'MENGE', 'MEINS']
    for col in numeric_candidates:
        if col in df_clean.columns:
            original_dtype = df_clean[col].dtype
            if df_clean[col].dtype == 'object':
                try:
                    # Try to convert to numeric
                    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
                    if verbose:
                        print(f"   ✅ Converted '{col}' from {original_dtype} to {df_clean[col].dtype}")
                except:
                    if verbose:
                        print(f"   ❌ Failed to convert '{col}' to numeric")
    
    # 5. Text Data Cleaning
    if verbose:
        print(f"\n📝 5. TEXT DATA CLEANING")
    
    text_columns = ['QMTXT', 'KURZTEXT', 'LTXTM', 'BZMNG']
    for col in text_columns:
        if col in df_clean.columns:
            original_nulls = df_clean[col].isnull().sum()
            # Strip whitespace and convert empty strings to NaN
            df_clean[col] = df_clean[col].astype(str).str.strip()
            df_clean[col] = df_clean[col].replace(['', 'nan', 'None', '0'], pd.NaT)
            new_nulls = df_clean[col].isnull().sum()
            if verbose and new_nulls != original_nulls:
                print(f"   📝 Cleaned '{col}': {new_nulls - original_nulls} empty values converted to NaN")
    
    # 6. Categorical Data Analysis
    if verbose:
        print(f"\n🏷️  6. CATEGORICAL DATA ANALYSIS")
    
    categorical_columns = ['QMART', 'PRIOK', 'QMGRP', 'QMCOD', 'MAWERK', 'LIFNR']
    for col in categorical_columns:
        if col in df_clean.columns:
            unique_count = df_clean[col].nunique()
            if verbose:
                print(f"   📊 '{col}': {unique_count} unique values")
                if unique_count <= 20:  # Show value counts for small categorical vars
                    print(f"      Values: {df_clean[col].value_counts(dropna=False).head().to_dict()}")
    
    # 7. Remove Duplicates
    if verbose:
        print(f"\n🗑️  7. DUPLICATE REMOVAL")
    
    initial_rows = len(df_clean)
    # Check for complete duplicates
    df_clean = df_clean.drop_duplicates()
    duplicates_removed = initial_rows - len(df_clean)
    cleaning_stats['duplicates_removed'] = duplicates_removed
    
    if verbose:
        if duplicates_removed > 0:
            print(f"   🗑️  Removed {duplicates_removed} duplicate rows")
        else:
            print(f"   ✅ No duplicate rows found")
    
    # 8. Final Statistics
    cleaning_stats.update({
        'final_rows': len(df_clean),
        'final_columns': len(df_clean.columns),
        'missing_values_after': df_clean.isnull().sum().sum(),
        'memory_mb': df_clean.memory_usage(deep=True).sum() / (1024**2)
    })
    
    if verbose:
        print(f"\n📈 8. CLEANING SUMMARY")
        print(f"=" * 40)
        print(f"Rows: {cleaning_stats['initial_rows']:,} → {cleaning_stats['final_rows']:,} "
              f"({cleaning_stats['final_rows'] - cleaning_stats['initial_rows']:+,})")
        print(f"Missing values: {cleaning_stats['missing_values_before']:,} → {cleaning_stats['missing_values_after']:,}")
        print(f"Date columns converted: {cleaning_stats['date_columns_converted']}")
        print(f"Memory usage: {cleaning_stats['memory_mb']:.2f} MB")
        print(f"Duplicates removed: {cleaning_stats['duplicates_removed']:,}")
    
    return df_clean, cleaning_stats

def convert_sap_date_column(df, column_name, verbose=True):
    """
    Convert SAP date column from YYYYMMDD format to datetime.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing the column
    column_name : str
        Name of the column to convert
    verbose : bool
        Whether to print conversion details
    
    Returns:
    --------
    dict : Conversion results and statistics
    """
    if column_name not in df.columns:
        return {'converted': False, 'reason': 'Column not found'}
    
    original_dtype = df[column_name].dtype
    original_nulls = df[column_name].isnull().sum()
    
    # Convert to string and handle SAP null representations
    df[column_name] = df[column_name].astype(str)
    
    # Replace SAP null representations
    sap_nulls = ['0', '0.0', '00000000', 'nan', 'None', '']
    df[column_name] = df[column_name].replace(sap_nulls, pd.NaT)
    
    # Convert to datetime
    df[column_name] = pd.to_datetime(df[column_name], format='%Y%m%d', errors='coerce')
    
    new_nulls = df[column_name].isnull().sum()
    conversion_failures = new_nulls - original_nulls
    
    if verbose:
        print(f"   📅 '{column_name}': {original_dtype} → {df[column_name].dtype}")
        if conversion_failures > 0:
            print(f"      ⚠️  {conversion_failures} values couldn't be converted")
        print(f"      📊 Valid dates: {len(df) - new_nulls:,}, Missing: {new_nulls:,}")
    
    return {
        'converted': True,
        'original_dtype': str(original_dtype),
        'new_dtype': str(df[column_name].dtype),
        'conversion_failures': conversion_failures,
        'total_nulls': new_nulls
    }

def analyze_missing_values(df):
    """
    Analyze missing values in the DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame to analyze
    
    Returns:
    --------
    pd.DataFrame : Missing value analysis
    """
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
    
    if missing_data.empty:
        return pd.DataFrame()
    
    missing_percent = (missing_data / len(df) * 100).round(2)
    
    missing_df = pd.DataFrame({
        'Missing_Count': missing_data,
        'Missing_Percentage': missing_percent
    })
    
    return missing_df

def get_qmel_quality_report(df_qmel_clean):
    """
    Generate a data quality report for cleaned QMEL data.
    
    Parameters:
    -----------
    df_qmel_clean : pd.DataFrame
        Cleaned QMEL DataFrame
    
    Returns:
    --------
    dict : Quality report
    """
    if df_qmel_clean.empty:
        return {'error': 'DataFrame is empty'}
    
    report = {
        'total_notifications': len(df_qmel_clean),
        'date_range': {},
        'notification_types': {},
        'priority_distribution': {},
        'data_completeness': {}
    }
    
    # Date range analysis
    if 'ERDAT' in df_qmel_clean.columns:
        valid_dates = df_qmel_clean['ERDAT'].dropna()
        if not valid_dates.empty:
            report['date_range'] = {
                'earliest': valid_dates.min(),
                'latest': valid_dates.max(),
                'span_days': (valid_dates.max() - valid_dates.min()).days
            }
    
    # Notification types
    if 'QMART' in df_qmel_clean.columns:
        report['notification_types'] = df_qmel_clean['QMART'].value_counts().head(10).to_dict()
    
    # Priority distribution
    if 'PRIOK' in df_qmel_clean.columns:
        report['priority_distribution'] = df_qmel_clean['PRIOK'].value_counts().to_dict()
    
    # Data completeness
    total_cells = df_qmel_clean.shape[0] * df_qmel_clean.shape[1]
    missing_cells = df_qmel_clean.isnull().sum().sum()
    report['data_completeness'] = {
        'completeness_percentage': round((1 - missing_cells/total_cells) * 100, 2),
        'missing_cells': missing_cells,
        'total_cells': total_cells
    }
    
    return report

# Usage example
if __name__ == "__main__":
    # Check if df_qmel exists and clean it
    if 'df_qmel' in globals() and not df_qmel.empty:
        print("🚀 Starting QMEL Data Cleaning...")
        
        # Clean the data
        df_qmel_clean, stats = clean_qmel_data(df_qmel, verbose=True)
        
        # Generate quality report
        print("\n" + "="*60)
        print("📋 QMEL DATA QUALITY REPORT")
        print("="*60)
        
        quality_report = get_qmel_quality_report(df_qmel_clean)
        
        print(f"📊 Total Notifications: {quality_report.get('total_notifications', 'N/A'):,}")
        
        if 'date_range' in quality_report and quality_report['date_range']:
            dr = quality_report['date_range']
            print(f"📅 Date Range: {dr['earliest'].strftime('%Y-%m-%d')} to {dr['latest'].strftime('%Y-%m-%d')}")
            print(f"   Span: {dr['span_days']:,} days")
        
        if 'notification_types' in quality_report and quality_report['notification_types']:
            print(f"🏷️  Top Notification Types:")
            for ntype, count in list(quality_report['notification_types'].items())[:5]:
                print(f"   {ntype}: {count:,}")
        
        if 'data_completeness' in quality_report:
            dc = quality_report['data_completeness']
            print(f"✅ Data Completeness: {dc['completeness_percentage']}%")
        
        print(f"\n🎯 Cleaned DataFrame 'df_qmel_clean' is ready for analysis!")
        
    else:
        print("❌ df_qmel not found or is empty. Please load the data first.")

🚀 Starting QMEL Data Cleaning...
🧹 STARTING QMEL DATA CLEANING PIPELINE
Initial dataset shape: (200, 165)

📊 1. INITIAL DATA ASSESSMENT
Shape: (200, 165)
Total missing values: 19,833
Memory usage: 0.36 MB

🔍 2. MISSING VALUES ANALYSIS
                        Missing_Count  Missing_Percentage
AENAM                             200               100.0
AUFNR                             200               100.0
WAERS                             200               100.0
VERID                             200               100.0
SA_AUFNR                          200               100.0
RM_WERKS                          200               100.0
RM_MATNR                          200               100.0
QMCOD                             200               100.0
AUSWIRK                           200               100.0
VKORG                             200               100.0
MATNR                             200               100.0
REVLV                             200               100.0
MATKL      

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def prepare_plant_data():
    """
    Create the plant description dataframe from your data
    """
    plant_data = {
        'Plant Code': ['A110', 'A111', 'A112', 'A113', 'A114', 'A210', 'A211', 
                      'A310', 'A410', 'A510', 'A610', 'A710', 'A810'],
        'Name': ['Plant1', 'Plant2', 'Plant3', 'Plant4', 'Plant5', 'Plant6', 
                'Plant7', 'Plant8', 'Plant9', 'Plant10', 'Plant11', 'Plant12', 'Plant13']
    }
    
    df_plant_description = pd.DataFrame(plant_data)
    
    # Also create SAP-standard column names as aliases
    df_plant_description['WERKS'] = df_plant_description['Plant Code']
    df_plant_description['NAME1'] = df_plant_description['Name']
    
    return df_plant_description

def create_comprehensive_view_corrected(df_aufk, df_afko=None, df_afpo=None, df_aufm=None, 
                                      df_qmel=None, df_qmfe=None, df_qmur=None, df_qmih=None, 
                                      df_qpcd=None, df_qpct=None, df_qpgt=None, df_crhd_v1=None, 
                                      df_jest=None, df_plant_description=None):
    """
    Creates a comprehensive view with corrected plant data handling
    """
    
    print("🔄 Building SAP Comprehensive View (CORRECTED)...")
    print("=" * 60)
    
    # Validate input data
    if df_aufk is None or df_aufk.empty:
        raise ValueError("Order Master (AUFK) data is required but empty!")
    
    print(f"📋 Starting with AUFK: {len(df_aufk):,} orders")
    print(f"   AUFK columns: {df_aufk.columns.tolist()}")
    
    # Step 1: Build Production Order Base
    print("\n📋 Step 1: Building Production Order Base...")
    base_df = df_aufk.copy()
    
    # Join with AFKO if available
    if df_afko is not None and not df_afko.empty:
        print(f"   AFKO columns: {df_afko.columns.tolist()}")
        
        # Find common columns for joining
        common_cols = list(set(df_aufk.columns) & set(df_afko.columns))
        
        if 'MANDT' in common_cols and 'AUFNR' in common_cols:
            join_keys = ['MANDT', 'AUFNR']
        elif 'AUFNR' in common_cols:
            join_keys = ['AUFNR']
        else:
            print(f"   ⚠️  No suitable join keys found between AUFK and AFKO")
            join_keys = None
        
        if join_keys:
            base_df = base_df.merge(df_afko, on=join_keys, how='left', suffixes=('', '_AFKO'))
            print(f"   ✓ Orders after AUFK + AFKO join: {len(base_df):,}")
    
    print(f"   Current base_df columns: {base_df.columns.tolist()}")
    
    # Step 2: Add Plant Information (CORRECTED)
    print("\n🏭 Step 2: Adding Plant Information...")
    
    if df_plant_description is None:
        # Create plant description from your data
        df_plant_description = prepare_plant_data()
        print("   ✓ Created plant description from provided data")
    
    print(f"   Plant description columns: {df_plant_description.columns.tolist()}")
    
    # Find plant-related columns in base_df
    possible_plant_cols = []
    for col in base_df.columns:
        col_upper = col.upper()
        if any(keyword in col_upper for keyword in ['WERK', 'PLANT', 'FACILITY']):
            possible_plant_cols.append(col)
    
    print(f"   Possible plant columns in base_df: {possible_plant_cols}")
    
    # Try different joining strategies
    plant_join_successful = False
    
    # Strategy 1: Standard SAP WERKS column
    if 'WERKS' in base_df.columns:
        try:
            if 'WERKS' in df_plant_description.columns:
                join_key = 'WERKS'
            elif 'Plant Code' in df_plant_description.columns:
                join_key = 'Plant Code'
                # Rename for consistency
                df_plant_description['WERKS'] = df_plant_description['Plant Code']
            
            base_df = base_df.merge(
                df_plant_description[['WERKS', 'Name'] if 'Name' in df_plant_description.columns else ['WERKS']],
                on='WERKS',
                how='left',
                suffixes=('', '_PLANT')
            )
            print(f"   ✓ Plant join successful on WERKS: {len(base_df):,}")
            plant_join_successful = True
        except Exception as e:
            print(f"   ❌ WERKS join failed: {e}")
    
    # Strategy 2: Try Plant Code directly
    if not plant_join_successful and 'Plant Code' in base_df.columns:
        try:
            base_df = base_df.merge(
                df_plant_description,
                on='Plant Code',
                how='left',
                suffixes=('', '_PLANT')
            )
            print(f"   ✓ Plant join successful on Plant Code: {len(base_df):,}")
            plant_join_successful = True
        except Exception as e:
            print(f"   ❌ Plant Code join failed: {e}")
    
    # Strategy 3: Try any other plant-related columns
    if not plant_join_successful:
        for col in possible_plant_cols:
            try:
                # Try joining with Plant Code
                base_df = base_df.merge(
                    df_plant_description,
                    left_on=col,
                    right_on='Plant Code',
                    how='left',
                    suffixes=('', '_PLANT')
                )
                print(f"   ✓ Plant join successful on {col} -> Plant Code: {len(base_df):,}")
                plant_join_successful = True
                break
            except Exception as e:
                print(f"   ❌ {col} -> Plant Code join failed: {e}")
    
    if not plant_join_successful:
        print(f"   ⚠️  No plant join successful. Available columns in base_df:")
        print(f"      {sorted(base_df.columns.tolist())}")
    
    # Step 3: Add Order Items (AFPO)
    print("\n📦 Step 3: Adding Order Items...")
    
    if df_afpo is not None and not df_afpo.empty:
        try:
            print(f"   AFPO columns: {df_afpo.columns.tolist()}")
            
            # Determine grouping columns
            if 'MANDT' in df_afpo.columns and 'AUFNR' in df_afpo.columns:
                group_cols = ['MANDT', 'AUFNR']
            elif 'AUFNR' in df_afpo.columns:
                group_cols = ['AUFNR']
            else:
                print(f"   ⚠️  No AUFNR column found in AFPO")
                group_cols = None
            
            if group_cols:
                # Create aggregation dictionary based on available columns
                agg_dict = {}
                
                if 'MATNR' in df_afpo.columns:
                    agg_dict['MATNR'] = lambda x: ', '.join(x.unique()[:5])
                if 'POSNR' in df_afpo.columns:
                    agg_dict['POSNR'] = 'count'
                elif 'AUFNR' in df_afpo.columns:
                    agg_dict['AUFNR_count'] = 'count'
                if 'CHARG' in df_afpo.columns:
                    agg_dict['CHARG'] = lambda x: ', '.join(x.dropna().unique()[:3])
                
                if agg_dict:
                    material_summary = df_afpo.groupby(group_cols).agg(agg_dict).reset_index()
                    
                    # Rename columns
                    rename_dict = {}
                    if 'MATNR' in agg_dict:
                        rename_dict['MATNR'] = 'ORDER_MATERIALS'
                    if 'POSNR' in agg_dict:
                        rename_dict['POSNR'] = 'ORDER_ITEM_COUNT'
                    if 'AUFNR_count' in agg_dict:
                        rename_dict['AUFNR_count'] = 'ORDER_ITEM_COUNT'
                    if 'CHARG' in agg_dict:
                        rename_dict['CHARG'] = 'ORDER_BATCHES'
                    
                    material_summary = material_summary.rename(columns=rename_dict)
                    
                    # Join back to base
                    base_df = base_df.merge(material_summary, on=group_cols, how='left')
                    print(f"   ✓ Order items added: {len(base_df):,}")
                else:
                    print(f"   ⚠️  No suitable aggregation columns found in AFPO")
            
        except Exception as e:
            print(f"   ❌ Order items processing failed: {e}")
    
    # Step 4: Add Quality Notifications (QMEL)
    print("\n🔍 Step 4: Adding Quality Notifications...")
    
    if df_qmel is not None and not df_qmel.empty:
        try:
            print(f"   QMEL columns: {df_qmel.columns.tolist()}")
            
            # Determine grouping columns
            if 'MANDT' in df_qmel.columns and 'AUFNR' in df_qmel.columns:
                group_cols = ['MANDT', 'AUFNR']
            elif 'AUFNR' in df_qmel.columns:
                group_cols = ['AUFNR']
            else:
                print(f"   ⚠️  No AUFNR column found in QMEL")
                group_cols = None
            
            if group_cols:
                # Create aggregation dictionary
                agg_dict = {}
                
                if 'QMNUM' in df_qmel.columns:
                    agg_dict['QMNUM'] = 'count'
                if 'QMART' in df_qmel.columns:
                    agg_dict['QMART'] = lambda x: ', '.join(x.unique())
                if 'KURZTEXT' in df_qmel.columns:
                    agg_dict['KURZTEXT'] = lambda x: ' | '.join(x.dropna().unique()[:3])
                
                if agg_dict:
                    quality_summary = df_qmel.groupby(group_cols).agg(agg_dict).reset_index()
                    
                    # Rename columns
                    rename_dict = {}
                    if 'QMNUM' in agg_dict:
                        rename_dict['QMNUM'] = 'QUALITY_NOTIF_COUNT'
                    if 'QMART' in agg_dict:
                        rename_dict['QMART'] = 'QUALITY_NOTIF_TYPES'
                    if 'KURZTEXT' in agg_dict:
                        rename_dict['KURZTEXT'] = 'QUALITY_ISSUES_DESC'
                    
                    quality_summary = quality_summary.rename(columns=rename_dict)
                    
                    # Join back to base
                    base_df = base_df.merge(quality_summary, on=group_cols, how='left')
                    print(f"   ✓ Quality notifications added: {len(base_df):,}")
                else:
                    print(f"   ⚠️  No suitable aggregation columns found in QMEL")
            
        except Exception as e:
            print(f"   ❌ Quality notifications processing failed: {e}")
    
    # Step 5: Add other tables (simplified for now)
    print("\n📊 Step 5: Processing other tables...")
    
    # Process other dataframes if provided
    other_tables = {
        'AUFM (Goods Movements)': df_aufm,
        'QMFE (Quality Defects)': df_qmfe,
        'QMUR (Root Causes)': df_qmur,
        'QMIH (Maintenance)': df_qmih,
        'CRHD_V1 (Work Centers)': df_crhd_v1
    }
    
    for table_name, df in other_tables.items():
        if df is not None and not df.empty:
            print(f"   📋 {table_name}: {len(df):,} rows, columns: {df.columns.tolist()}")
        else:
            print(f"   📋 {table_name}: Not provided or empty")
    
    # Step 6: Create derived fields
    print("\n🛠️ Step 6: Creating Derived Fields...")
    
    # Fill NaN values for numeric columns
    numeric_cols = ['QUALITY_NOTIF_COUNT', 'ORDER_ITEM_COUNT']
    
    for col in numeric_cols:
        if col in base_df.columns:
            base_df[col] = base_df[col].fillna(0).astype(int)
    
    # Create quality indicators
    if 'QUALITY_NOTIF_COUNT' in base_df.columns:
        base_df['HAS_QUALITY_ISSUES'] = (base_df['QUALITY_NOTIF_COUNT'] > 0)
        
        # Simple quality score (0-100, higher is better)
        max_issues = max(base_df['QUALITY_NOTIF_COUNT'].max(), 1)
        base_df['QUALITY_SCORE'] = 100 - (base_df['QUALITY_NOTIF_COUNT'] / max_issues * 100)
        base_df['QUALITY_SCORE'] = base_df['QUALITY_SCORE'].clip(0, 100).round(1)
        
        # Quality category
        base_df['QUALITY_CATEGORY'] = pd.cut(
            base_df['QUALITY_SCORE'],
            bins=[0, 60, 80, 100],
            labels=['Poor', 'Good', 'Excellent'],
            include_lowest=True
        )
        
        print(f"   ✓ Quality indicators created")
    
    print(f"   ✓ Final comprehensive dataset: {len(base_df):,} records")
    print(f"   ✓ Final columns ({len(base_df.columns)}): {base_df.columns.tolist()}")
    
    # Generate Summary Statistics
    print("\n📊 Step 7: Generating Summary Statistics...")
    
    summary_stats = {
        'total_orders': len(base_df),
        'total_columns': len(base_df.columns),
        'column_names': base_df.columns.tolist()
    }
    
    # Plant analysis
    plant_col = None
    for col in ['WERKS', 'Plant Code', 'Plant_Code']:
        if col in base_df.columns:
            plant_col = col
            break
    
    if plant_col:
        summary_stats['total_plants'] = base_df[plant_col].nunique()
        summary_stats['plant_distribution'] = base_df[plant_col].value_counts().head(10).to_dict()
    
    # Quality analysis
    if 'HAS_QUALITY_ISSUES' in base_df.columns:
        summary_stats['orders_with_quality_issues'] = base_df['HAS_QUALITY_ISSUES'].sum()
        summary_stats['quality_issue_percentage'] = (summary_stats['orders_with_quality_issues'] / len(base_df)) * 100
    
    if 'QUALITY_SCORE' in base_df.columns:
        summary_stats['avg_quality_score'] = base_df['QUALITY_SCORE'].mean()
        summary_stats['min_quality_score'] = base_df['QUALITY_SCORE'].min()
        summary_stats['max_quality_score'] = base_df['QUALITY_SCORE'].max()
    
    if 'QUALITY_CATEGORY' in base_df.columns:
        summary_stats['quality_distribution'] = base_df['QUALITY_CATEGORY'].value_counts().to_dict()
    
    print("\n✅ Comprehensive View Created Successfully!")
    print("=" * 60)
    print(f"📈 Summary Statistics:")
    print(f"   • Total Orders: {summary_stats['total_orders']:,}")
    print(f"   • Total Columns: {summary_stats['total_columns']}")
    
    if 'total_plants' in summary_stats:
        print(f"   • Total Plants: {summary_stats['total_plants']}")
    
    if 'orders_with_quality_issues' in summary_stats:
        print(f"   • Orders with Quality Issues: {summary_stats['orders_with_quality_issues']:,} ({summary_stats['quality_issue_percentage']:.1f}%)")
    
    if 'avg_quality_score' in summary_stats:
        print(f"   • Average Quality Score: {summary_stats['avg_quality_score']:.1f}/100")
    
    return base_df, summary_stats

def analyze_results(comprehensive_df, summary_stats):
    """
    Analyze the comprehensive view results
    """
    print("\n🔍 DETAILED ANALYSIS")
    print("=" * 60)
    
    # Basic info
    print(f"\n📊 Dataset Overview:")
    print(f"   • Shape: {comprehensive_df.shape}")
    print(f"   • Memory usage: {comprehensive_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Column analysis
    print(f"\n📋 Column Analysis:")
    for col in comprehensive_df.columns:
        non_null = comprehensive_df[col].notna().sum()
        null_pct = (len(comprehensive_df) - non_null) / len(comprehensive_df) * 100
        print(f"   • {col}: {non_null:,} non-null ({100-null_pct:.1f}%)")
    
    # Plant analysis
    plant_cols = [col for col in comprehensive_df.columns if any(keyword in col.upper() for keyword in ['WERK', 'PLANT'])]
    if plant_cols:
        print(f"\n🏭 Plant Analysis:")
        for col in plant_cols:
            if comprehensive_df[col].dtype == 'object':
                print(f"   • {col}: {comprehensive_df[col].value_counts().head().to_dict()}")
    
    # Quality analysis
    if 'QUALITY_CATEGORY' in comprehensive_df.columns:
        print(f"\n🔍 Quality Analysis:")
        quality_dist = comprehensive_df['QUALITY_CATEGORY'].value_counts()
        for category, count in quality_dist.items():
            pct = count/len(comprehensive_df)*100
            print(f"   • {category}: {count:,} orders ({pct:.1f}%)")
    
    return True

# Usage example
def main():
    """
    Main function demonstrating usage
    """
    print("🚀 SAP Quality Management Data Integration")
    print("=" * 60)
    print("\nTo use this script with your data:")
    print("\n1. Load your SAP data:")
    print("   df_aufk = pd.read_csv('your_aufk_file.csv')")
    print("   df_afko = pd.read_csv('your_afko_file.csv')")
    print("   # ... load other files")
    
    print("\n2. Run the integration:")
    print("   comprehensive_df, summary = create_comprehensive_view_corrected(")
    print("       df_aufk=df_aufk,")
    print("       df_afko=df_afko,")
    print("       # ... other dataframes")
    print("   )")
    
    print("\n3. Analyze results:")
    print("   analyze_results(comprehensive_df, summary)")
    
    print("\n4. Export results:")
    print("   comprehensive_df.to_csv('sap_comprehensive_view.csv', index=False)")
    print("   comprehensive_df.to_excel('sap_comprehensive_view.xlsx', index=False)")

if __name__ == "__main__":
    main()

🚀 SAP Quality Management Data Integration

To use this script with your data:

1. Load your SAP data:
   df_aufk = pd.read_csv('your_aufk_file.csv')
   df_afko = pd.read_csv('your_afko_file.csv')
   # ... load other files

2. Run the integration:
   comprehensive_df, summary = create_comprehensive_view_corrected(
       df_aufk=df_aufk,
       df_afko=df_afko,
       # ... other dataframes
   )

3. Analyze results:
   analyze_results(comprehensive_df, summary)

4. Export results:
   comprehensive_df.to_csv('sap_comprehensive_view.csv', index=False)
   comprehensive_df.to_excel('sap_comprehensive_view.xlsx', index=False)


In [15]:
comprehensive_df, summary = create_comprehensive_view(
       df_aufk, df_afko, df_afpo, df_aufm, df_qmel, df_qmfe,
       df_qmur, df_qmih, df_qpcd, df_qpct, df_qpgt,
       df_crhd_v1, df_jest, df_plant_description)

🔄 Building SAP Comprehensive View...
📋 Step 1: Building Production Order Base...
   ✓ Orders after AUFK + AFKO join: 3
   ✓ Orders after Work Center join: 3


KeyError: 'WERKS'